In [2]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import warnings
import time
from collections import Counter
warnings.filterwarnings("ignore")


plt.style.use("fivethirtyeight")

sns.set_style({'font.sans-serif': ['simsun', 'Arial']})
sns.set_style('darkgrid', {'font.sans-serif': ['simhei', 'Arial']})
%matplotlib inline
current_date = pd.to_datetime(
    time.strftime('%Y-%m-%d', time.localtime(time.time())))

res = {}
data_path = r'../../.././Documents/赛题1数据集/'
current_date

Timestamp('2019-04-05 00:00:00')

# 资产负债表

In [35]:
df_asset_bpk = pd.read_excel(data_path+r'上市信息财务信息资产负债表.xlsx')
df_asset = df_asset_bpk.copy()

In [92]:
A = ['资产:货币资金(元)', '资产:长期股权投资(元)', '资产:固定资产(元)', '资产:无形资产(元)','负债:长期负债合计(元)','权益:实收资本(或股本)(元)',
     '资产:流动资产合计(元)','资产:资产总计(元)','负债:流动负债合计(元)','负债:负债合计(元)','权益:股东权益合计(元)']
df_asset=df_asset[~df_asset['权益:股东权益合计(元)'].isin(['正无穷大万亿'])]
df_asset1= pd.DataFrame({'企业编号':df_asset['企业编号'],'资产:货币资金(元)':df_asset['资产:货币资金(元)'],'资产:长期股权投资(元)': df_asset['资产:长期股权投资(元)'],
                         '资产:固定资产(元)': df_asset['资产:固定资产(元)'],'资产:无形资产(元)': df_asset['资产:无形资产(元)'],'负债:长期负债合计(元)':df_asset['负债:长期负债合计(元)'],
                         '权益:实收资本(或股本)(元)':df_asset['权益:实收资本(或股本)(元)'],'资产:流动资产合计(元)': df_asset['资产:流动资产合计(元)'],
                         '资产:资产总计(元)': df_asset['资产:资产总计(元)'],'负债:流动负债合计(元)': df_asset['负债:流动负债合计(元)'],'负债:负债合计(元)':df_asset['负债:负债合计(元)'],
                         '权益:股东权益合计(元)':df_asset['权益:股东权益合计(元)'],
})



In [93]:
# for arr in A:
#     chinese = set(df_asset1[arr].str.extract(
#         "([^0-9\.\-]+)", expand=False).dropna())
#     print(arr, chinese)
for arr in A:
    df_asset1.loc[(df_asset1[arr].str.contains("万") == True), arr] = df_asset1.loc[(
        df_asset1[arr].str.contains("万") == True), arr].str.replace('万', '').astype(float)*0.0001
    df_asset1.loc[(df_asset1[arr].str.contains("亿") == True), arr] = df_asset1.loc[(
        df_asset1[arr].str.contains("亿") == True), arr].str.replace('亿', '').astype(float)*1
for arr in A:
     df_asset1[arr]=pd.to_numeric( df_asset1[arr],errors='coerce')

In [108]:
df_asset1.loc[(df_asset1['资产:货币资金(元)'] >1718) ==True,'资产:货币资金(元)']=1718
df_asset1.loc[(df_asset1['资产:长期股权投资(元)'] >5700) ==True,'资产:长期股权投资(元)']=5700
df_asset1.loc[(df_asset1['资产:固定资产(元)'] >6900) ==True,'资产:固定资产(元)']=6900
df_asset1.loc[(df_asset1['资产:无形资产(元)'] >1800) ==True,'资产:无形资产(元)']=1800
df_asset1.loc[(df_asset1['负债:长期负债合计(元)'] >3994) ==True,'负债:长期负债合计(元)']=3994
df_asset1.loc[(df_asset1['权益:实收资本(或股本)(元)'] >3564) ==True,'权益:实收资本(或股本)(元)']=3564
df_asset1.loc[(df_asset1['资产:流动资产合计(元)'] >13264) ==True,'资产:流动资产合计(元)']=13264
df_asset1.loc[(df_asset1['资产:资产总计(元)'] >38981) ==True,'资产:资产总计(元)']=38981
df_asset1.loc[(df_asset1['负债:流动负债合计(元)'] >10441) ==True,'负债:流动负债合计(元)']=10441
df_asset1.loc[(df_asset1['负债:负债合计(元)'] >259172) ==True,'负债:负债合计(元)']=259172
df_asset1.loc[(df_asset1['权益:股东权益合计(元)'] >22808) ==True,'权益:股东权益合计(元)']=22808

    



In [112]:
agg_func1 ={'资产:流动资产合计(元)': ['mean', 'max', 'min'],
    '资产:资产总计(元)': ['mean', 'max', 'min'],
    '负债:流动负债合计(元)': ['mean', 'max', 'min'],
    '负债:负债合计(元)': ['mean', 'max', 'min'],
    '权益:股东权益合计(元)':['mean', 'max', 'min'],
     '资产:货币资金(元)': ['mean', 'max', 'min'],
     '资产:长期股权投资(元)': ['mean', 'max', 'min'],
    '资产:固定资产(元)': ['mean', 'max', 'min'],
    '资产:无形资产(元)': ['mean', 'max', 'min'],
    '权益:实收资本(或股本)(元)':['mean', 'max', 'min'],}
df_asset1_new = df_asset1.groupby(['企业编号']).agg(agg_func1)
df_asset1_new.columns = ['_'.join(col) for col in df_asset1_new.columns.values]
df_asset1_new.columns = [
    '资产:流动资产合计(元)_mean', '资产:流动资产合计(元)_max', '资产:流动资产合计(元)_min','资产:资产总计(元)_mean', '资产:资产总计(元)_max', '资产:资产总计(元)_min', 
     '负债:流动负债合计(元)_mean', '负债:流动负债合计(元)_max', '负债:流动负债合计(元)_min', '负债:负债合计(元)_mean', '负债:负债合计(元)_max', '负债:负债合计(元)_min',
     '权益:股东权益合计(元)_mean', '权益:股东权益合计(元)_max', '权益:股东权益合计(元)_min','资产:货币资金(元)_mean', '资产:货币资金(元)_max', '资产:货币资金(元)_min',
    '资产:长期股权投资(元)_mean', '资产:长期股权投资(元)_max', '资产:长期股权投资(元)_min', '资产:固定资产(元)_mean', '资产:固定资产(元)_max', '资产:固定资产(元)_min', 
    '资产:无形资产(元)_mean', '资产:无形资产(元)_max', '资产:无形资产(元)_min','权益:实收资本(或股本)(元)_mean', '权益:实收资本(或股本)(元)_max', '权益:实收资本(或股本)(元)_min' ]
df_asset1_new['企业编号'] = df_asset1_new.index
df_asset1_new.fillna(df_asset1_new.mean(),inplace=True)


In [118]:
res['资产负债表']=df_asset1_new

# 利润表

In [3]:
df_profit_bpk = pd.read_excel(data_path+r'上市信息财务信息-利润表.xlsx')
df_profit = df_profit_bpk.copy()
B=['营业成本(元)','销售费用(元)','财务费用(元)','管理费用(元)','投资收益(元)','营业利润(元)','利润总额(元)','归属母公司所有者净利润(元)']
# for arr in B:
#     chinese = set(df_profit[arr].str.extract(
#         "([^0-9\.\-]+)", expand=False).dropna())
#     print(arr, chinese)
for arr in B:
    df_profit.loc[(df_profit[arr].str.contains("万") == True), arr] = df_profit.loc[(
       df_profit[arr].str.contains("万") == True), arr].str.replace('万', '').astype(float)*10000
    df_profit.loc[(df_profit[arr].str.contains("亿") == True), arr] = df_profit.loc[(
        df_profit[arr].str.contains("亿") == True), arr].str.replace('亿', '').astype(float)*100000000
    df_profit[arr]=pd.to_numeric(df_profit[arr],errors='coerce')
df_profit1=df_profit.drop(['标题','日期','营业收入(元)','资产减值损失(元)','资产减值损失(元)','所得税(元)'],axis=1)



In [4]:
agg_func2 = {
     '营业成本(元)':  ['mean', 'max', 'min'],
    '营业利润(元)': ['mean', 'max', 'min'], '利润总额(元)':['mean', 'max', 'min'],
     '销售费用(元)': ['mean', 'max', 'min'],
    '财务费用(元)':  ['mean', 'max', 'min'],
    '管理费用(元)': ['mean', 'max', 'min'],
    '投资收益(元)': ['mean', 'max', 'min'],
}
df_profit1_new= df_profit1.groupby(['企业编号']).agg(agg_func2)
df_profit1_new.columns = ['_'.join(col) for col in df_profit1_new.columns.values]
df_profit1_new.columns = [
'营业成本(元)_mean', '营业成本(元)_max', '营业成本(元)_min','营业利润(元)_mean', '营业利润(元)_max', '营业利润(元)_min', 
     '利润总额(元)_mean', '利润总额(元)_max', '利润总额(元)_min', '销售费用(元)_mean', '销售费用(元)_max', '销售费用(元)_min','财务费用(元)_mean', '财务费用(元)_max', '财务费用(元)_min', 
     '管理费用(元)_mean', '管理费用(元)_max', '管理费用(元)_min', '投资收益(元)_mean', '投资收益(元)_max', '投资收益(元)_min',]
df_profit1_new['企业编号'] = df_profit1_new.index
df_profit1_new.fillna(df_profit1_new.mean(),inplace=True)


In [5]:
res['利润表']=df_profit1_new

# 现金流量表

In [65]:
df_cash_bpk = pd.read_excel(data_path+r'上市信息财务信息-现金流量表.xlsx')
df_cash = df_cash_bpk.copy()

In [119]:
df_cash1=pd.DataFrame({'企业编号':df_cash['企业编号'],'经营流入':df_cash['经营:经营活动现金流入小计(元)'],
                '经营净额':df_cash['经营:经营活动产生的现金流量净额(元)'], '投资流出':df_cash['投资:投资活动现金流出小计(元)'], '投资净额':df_cash['投资:投资活动产生的现金流量净额(元)'],
                  '筹资流入':df_cash['筹资:筹资活动现金流入小计(元)'],'筹资净额':df_cash['筹资活动产生的现金流量净额(元)'],})
C=['经营流入','经营净额','投资流出','投资净额', '筹资流入','筹资净额']
# for arr in C:
#     chinese = set(df_cash1[arr].str.extract(
#         "([^0-9\.\-]+)", expand=False).dropna())
#     print(arr, chinese)
for arr in C:
    df_cash1.loc[(df_cash1[arr].str.contains("万") == True), arr] = df_cash1.loc[(
        df_cash1[arr].str.contains("万") == True), arr].str.replace('万', '').astype(float)*0.0001
    df_cash1.loc[(df_cash1[arr].str.contains("亿") == True), arr] = df_cash1.loc[(
        df_cash1[arr].str.contains("亿") == True), arr].str.replace('亿', '').astype(float)*1
    df_cash1[arr]=pd.to_numeric(df_cash1[arr],errors='coerce')



In [122]:
agg_func3 = {
     '经营流入':  ['mean', 'max', 'min'],
    '经营净额': ['mean', 'max', 'min'], '投资流出':['mean', 'max', 'min'],'投资净额': ['mean', 'max', 'min'],
     '筹资流入': ['mean', 'max', 'min'],
    '筹资净额':  ['mean', 'max', 'min'],
}
df_cash1_new= df_cash1.groupby(['企业编号']).agg(agg_func3)
df_cash1_new.columns = ['_'.join(col) for col in df_cash1_new.columns.values]
df_cash1_new.columns = [
'经营流入_mean', '经营流入_max', '经营流入_min','经营净额_mean', '经营净额_max', '经营净额_min', 
     '投资流出_mean', '投资流出_max', '投资流出_min', '投资净额_mean', '投资净额_max',
    '投资净额_min','筹资流入_mean', '筹资流入_max', '筹资流入_min','筹资净额_mean', '筹资净额_max', '筹资净额_min']
df_cash1_new['企业编号'] =df_cash1_new.index
df_cash1_new.fillna(df_cash1_new.mean(),inplace=True)

In [126]:
res['现金流量表']=df_cash1_new

# 每股指标

In [127]:
df_meiguzhibiao_bpk = pd.read_excel(data_path+r'上市公司财务信息-每股指标.xlsx')
df_meiguzhibiao = df_meiguzhibiao_bpk.copy()
df_meiguzhibiao1=df_meiguzhibiao.drop(['标题','日期','稀释每股收益(元)'],axis=1)
D=['基本每股收益(元)','每股净资产(元)','每股公积金(元)','每股未分配利润(元)','每股经营现金流(元)']
for arr in D:
    df_meiguzhibiao1[arr]=pd.to_numeric(df_meiguzhibiao1[arr],errors='coerce')


In [128]:
agg_func4 = {
     '基本每股收益(元)':  ['mean', 'max', 'min'],
    '每股净资产(元)':['mean', 'max', 'min'],
     '每股未分配利润(元)': ['mean', 'max', 'min'],
    '每股经营现金流(元)':  ['mean', 'max', 'min'],
}
df_meiguzhibiao1_new= df_meiguzhibiao1.groupby(['企业编号']).agg(agg_func4)
df_meiguzhibiao1_new.columns = ['_'.join(col) for col in df_meiguzhibiao1_new.columns.values]
df_meiguzhibiao1_new.columns = [
'基本每股收益(元)_mean', '基本每股收益(元)_max', '基本每股收益(元)_min','每股净资产(元)_mean', '每股净资产(元)_max', '每股净资产(元)_min', 
     '每股未分配利润(元)_mean', '每股未分配利润(元)_max', '每股未分配利润(元)_min', '每股经营现金流(元)_mean', '每股经营现金流(元)_max',
    '每股经营现金流(元)_min']
df_meiguzhibiao1_new['企业编号'] =df_meiguzhibiao1_new.index
df_meiguzhibiao1_new.fillna(df_meiguzhibiao1_new.mean(),inplace=True)


In [129]:
res['每股指标']=df_meiguzhibiao1_new

# 成长能力指标

In [8]:
df_growth_bpk = pd.read_excel(data_path+r'上市信息财务信息-成长能力指标.xlsx')
df_growth= df_growth_bpk.copy()
df_growth=df_growth.drop(['标题','日期','扣非净利润(元)','归属净利润同比增长(元)','营业总收入滚动环比增长(元)', '归属净利润滚动环比增长(元)', '扣非净利润滚动环比增长(元)'],axis=1)
E = ['营业总收入(元)', '毛利润(元)', '归属净利润(元)']
for arr in E:
    df_growth.loc[(df_growth[arr].str.contains("万亿") == True), arr] = df_growth.loc[(
        df_growth[arr].str.contains("万亿") == True), arr].str.replace('万亿', '').astype(float)*1000000000000
    df_growth.loc[(df_growth[arr].str.contains("万") == True), arr] = df_growth.loc[(
        df_growth[arr].str.contains("万") == True), arr].str.replace('万', '').astype(float)*10000
    df_growth.loc[(df_growth[arr].str.contains("亿") == True), arr] = df_growth.loc[(
        df_growth[arr].str.contains("亿") == True), arr].str.replace('亿', '').astype(float)*100000000
F = ['营业总收入同比增长(元)', '扣非净利润同比增长(元)']
for arr in F:
    df_growth[arr] = df_growth[arr].replace('--%', np.nan)
for arr in F:
    df_growth.loc[(df_growth[arr].str.contains("%") == True), arr] = df_growth.loc[(
        df_growth[arr].str.contains("%") == True), arr].str.replace(',', '').str.rstrip('%').astype(float)/100
df_growth = df_growth.convert_objects(convert_numeric=True)

In [9]:
agg_func5= {
     '营业总收入(元)':  ['mean', 'max', 'min'],
     '归属净利润(元)':['mean', 'max', 'min'],
     '营业总收入同比增长(元)': ['mean', 'max', 'min'],
   '扣非净利润同比增长(元)':  ['mean', 'max', 'min'],'毛利润(元)': ['mean', 'max', 'min'],
}
df_growth_new= df_growth.groupby(['企业编号']).agg(agg_func5)
df_growth_new.columns = ['_'.join(col) for col in df_growth_new.columns.values]
df_growth_new.columns = [
'营业总收入(元)_mean', '营业总收入(元)_max', '营业总收入(元)_min','归属净利润(元)_mean', '归属净利润(元)_max', '归属净利润(元)_min', 
     '营业总收入同比增长(元)_mean', '营业总收入同比增长(元)_max', '营业总收入同比增长(元)_min', '扣非净利润同比增长(元)_mean', '扣非净利润同比增长(元)_max',
    '扣非净利润同比增长(元)_min','毛利润(元)_mean', '毛利润(元)_max', '毛利润(元)_min']
df_growth_new['企业编号'] =df_growth_new.index
df_growth_new.fillna(df_growth_new.mean(),inplace=True)



In [10]:
res['成长能力']=df_growth_new

# 盈利能力指标

In [11]:
df_abilityofprofit_bpk= pd.read_excel(data_path+r'上市信息财务信息盈利能力指标.xlsx')
df_abilityofprofit= df_abilityofprofit_bpk.copy()
G=['加权净资产收益率(%)','摊薄净资产收益率(%)','摊薄总资产收益率(%)','毛利率(%)','净利率(%)','实际税率(%)']
for arr in G:
        df_abilityofprofit[arr] = df_abilityofprofit[arr].replace('--%', np.nan)
        df_abilityofprofit[arr] =df_abilityofprofit[arr].str.rstrip('%').astype(float)/100
df_abilityofprofit.convert_objects(convert_numeric=True)
df_abilityofprofit=df_abilityofprofit.drop(['标题','日期'],axis=1)

In [13]:
agg_func6= {
     '加权净资产收益率(%)':  ['mean', 'max', 'min'],
   '毛利率(%)':['mean', 'max', 'min'],'净利率(%)': ['mean', 'max', 'min'],'实际税率(%)': ['mean', 'max', 'min'],
    '摊薄总资产收益率(%)': ['mean', 'max', 'min'],
}
df_abilityofprofit_new= df_abilityofprofit.groupby(['企业编号']).agg(agg_func6)
df_abilityofprofit_new.columns = ['_'.join(col) for col in df_abilityofprofit_new.columns.values]
df_abilityofprofit_new.columns = [
'加权净资产收益率(%)_mean', '加权净资产收益率(%)_max', '加权净资产收益率(%)_min',
'毛利率(%)_mean', '毛利率(%)_max', '毛利率(%)_min', '净利率(%)_mean', '净利率(%)_max','净利率(%)_min',
    '实际税率(%)_mean', '实际税率(%)_max', '实际税率(%)_min', '摊薄总资产收益率(%)_mean', '摊薄总资产收益率(%)_max', 
    '摊薄总资产收益率(%)_min']
df_abilityofprofit_new['企业编号'] =df_abilityofprofit_new.index
df_abilityofprofit_new.fillna(df_abilityofprofit_new.mean(),inplace=True)


In [14]:
res['盈利能力能力']=df_abilityofprofit_new

# 财务风险指标

In [178]:
df_risk_bpk= pd.read_excel(data_path+r'上市信息财务信息-财务风险指标.xlsx')
df_risk= df_risk_bpk.copy()
H=['流动比率','速动比率']
for arr in H:
      df_risk[arr] = pd.to_numeric(df_risk[arr],errors='coerce')
I=['资产负债率(%)','流动负债/总负债(%)']
for arr in I:
     df_risk[arr] = df_risk[arr].replace('--%', np.nan)
df_risk.convert_objects(convert_numeric=True)
for arr in I:
    df_risk[arr] =df_risk[arr].str.rstrip('%').astype(float)/100
df_risk.convert_objects(convert_numeric=True)
df_risk=df_risk.drop(['标题','日期','速动比率'],axis=1)
df_risk=df_risk[~df_risk['流动比率'].isin([103.22])]
df_risk=df_risk[~df_risk['资产负债率(%)'].isin([5.6089])]
df_risk=df_risk[~df_risk['资产负债率(%)'].isin([4.5429])]

In [161]:
agg_func7= {
     '资产负债率(%)':  ['mean', 'max', 'min'],
     '流动负债/总负债(%)':['mean', 'max', 'min'],
     '流动比率': ['mean', 'max', 'min'],
}
df_risk_new= df_risk.groupby(['企业编号']).agg(agg_func7)
df_risk_new.columns = ['_'.join(col) for col in df_risk_new.columns.values]
df_risk_new.columns = [
'资产负债率_mean', '资产负债率_max', '资产负债率_min','流动负债/总负债_mean', '流动负债/总负债_max', '流动负债/总负债_min', 
 '流动比率_mean', '流动比率_max', '流动比率_min']
df_risk_new['企业编号'] =df_risk_new.index
df_risk_new.fillna(df_risk_new.mean(),inplace=True)


In [162]:
res['财务风险能力']=df_risk_new

# 运营能力指标

In [9]:
df_operation_bpk= pd.read_excel(data_path+r'上市信息财务信息运营能力指标.xlsx')
df_operation= df_operation_bpk.copy()
J=['应收账款周转天数(天)','存货周转天数(天)']
for arr in J:
    df_operation[arr]=pd.to_numeric(df_operation[arr],errors='coerce')
df_operation=df_operation.drop(['标题','日期'],axis=1)
df_operation=df_operation[~df_operation['应收账款周转天数(天)'].isin([47789.4])]
df_operation=df_operation[~df_operation['应收账款周转天数(天)'].isin([176659.70])]


In [10]:
agg_func8= {
     '总资产周转率(次)':  ['mean', 'max', 'min'],
     '应收账款周转天数(天)':['mean', 'max', 'min'],
     '存货周转天数(天)': ['mean', 'max', 'min'],
}
df_operation_new= df_operation.groupby(['企业编号']).agg(agg_func8)
df_operation_new.columns = ['_'.join(col) for col in df_operation_new.columns.values]
df_operation_new.columns = [
'总资产周转率(次)_mean', '总资产周转率(次)_max', '总资产周转率(次)_min','应收账款周转天数(天)_mean', '应收账款周转天数(天)_max', '应收账款周转天数(天)_min', 
 '存货周转天数(天)_mean', '存货周转天数(天)_max', '存货周转天数(天)_min']
df_operation_new['企业编号'] =df_operation_new.index
df_operation_new.fillna(df_operation_new.mean(),inplace=True)



In [11]:
res['运营能力指标']=df_operation_new